In [1]:
import util.utils as utils

percentage = 0.02
rand_seed = 42
data_sampling_freq = '10s'
rolling_window_size = '10s'

df_result = utils.get_binary(percentage, rand_seed)
df_audio = utils.get_audio(percentage, rand_seed, data_sampling_freq, rolling_window_size)

display(df_audio)
display(df_result)

AUDIO_AMPLITUDE  FORMANT_F1   FORMANT_F2   FORMANT_F3  \
ID  TIMESTAMP                                                                
386 0 days 00:00:00        -0.000004  614.645050  1569.856156  2555.688481   
    0 days 00:00:10        -0.000022  723.162229  1729.766756  2606.151056   
    0 days 00:00:20        -0.000004  612.055450  1654.936994  2564.859750   
    0 days 00:00:30        -0.000016  536.626819  1658.906150  2599.293833   
    0 days 00:00:40        -0.000023  470.114869  1632.436436  2606.598722   
...                              ...         ...          ...          ...   
391 0 days 00:10:40        -0.000037  601.069346  1592.053147  2508.186500   
    0 days 00:10:50        -0.000004  758.848328  1704.821917  2577.518944   
    0 days 00:11:00        -0.000042  827.604836  1755.064408  2635.432333   
    0 days 00:11:10         0.000012  758.418220  1721.411781  2655.801250   
    0 days 00:11:20         0.000039  380.222222  1583.388889  2353.888889   

                      FORMANT_F4   FORMANT_F5  COVAREP_F0  COVAREP_VUV  \
ID  TIMESTAMP                                                            
386 0 days 00:00:00  3448.281500  4355.226917  244.852083     0.135556   
    0 days 00:00:10  3481.564889  4401.983833  250.858750     0.044167   
    0 days 00:00:20  3452.678694  4339.918194  248.480833     0.446944   
    0 days 00:00:30  3510.076361  4367.084278  234.161667     0.433056   
    0 days 00:00:40  3513.154611  4338.591056  220.591250     0.512778   
...                          ...          ...         ...          ...   
391 0 days 00:10:40  3406.220000  4326.081000  254.759028     0.341944   
    0 days 00:10:50  3480.200111  4356.791861  271.011250     0.148611   
    0 days 00:11:00  3491.444111  4389.154861  274.051389     0.044167   
    0 days 00:11:10  3507.398111  4381.977750  272.903333     0.209167   
    0 days 00:11:20  3300.444444  4439.611111   73.000000     0.000000   

                     COVAREP_NAQ  COVAREP_QOQ  ...  COVAREP_HMPDD_3  \
ID  TIMESTAMP                                  ...                    
386 0 days 00:00:00     0.056976     0.150301  ...        -1.064362   
    0 days 00:00:10     0.042157     0.103817  ...        -1.052944   
    0 days 00:00:20     0.058107     0.177877  ...        -1.153529   
    0 days 00:00:30     0.057946     0.173296  ...        -1.110738   
    0 days 00:00:40     0.061950     0.199593  ...        -1.129264   
...                          ...          ...  ...              ...   
391 0 days 00:10:40     0.065125     0.221965  ...        -1.160749   
    0 days 00:10:50     0.037315     0.155814  ...        -1.129113   
    0 days 00:11:00     0.020857     0.069702  ...        -1.142678   
    0 days 00:11:10     0.049178     0.140093  ...        -1.189404   
    0 days 00:11:20     0.000000     0.000000  ...        -0.116297   

                     COVAREP_HMPDD_4  COVAREP_HMPDD_5  COVAREP_HMPDD_6  \
ID  TIMESTAMP                                                            
386 0 days 00:00:00        -0.912175        -0.759586        -0.614320   
    0 days 00:00:10        -0.925264        -0.766965        -0.607162   
    0 days 00:00:20        -0.958603        -0.744215        -0.546716   
    0 days 00:00:30        -0.951310        -0.752610        -0.588919   
    0 days 00:00:40        -0.958322        -0.773844        -0.608530   
...                              ...              ...              ...   
391 0 days 00:10:40        -0.953898        -0.753846        -0.576185   
    0 days 00:10:50        -0.934019        -0.743022        -0.564444   
    0 days 00:11:00        -0.958067        -0.770274        -0.606131   
    0 days 00:11:10        -0.987858        -0.776655        -0.579951   
    0 days 00:11:20        -0.052773        -0.076418        -0.097877   

                     COVAREP_HMPDD_7  COVAREP_HMPDD_8  COVAREP_HMPDD_9  \
ID  TIMESTAMP                                                            
386 0 da

,PHQ_Binary
ID,
386,1
391,0


### EDA

In [ ]:
# functional frequency / energy distribution


In [ ]:
# MFCC


In [0]:
# speed of speech (if available)


### train test split

In [ ]:
# merge
import pandas as pd

df = pd.merge(df_audio, df_result, on='ID')

In [ ]:
# split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)


### pipeline & hyperparameter tuning

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# creating pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor())
])

# hyperparameter tuning
# Define parameter grid for tuning
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5, 10]
}

# Use GridSearchCV with TimeSeriesSplit cross-validation
grid_search = GridSearchCV(
    pipeline, param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1
)

# Fit grid search to the data
grid_search.fit(X, y)

### evaluation  

In [12]:
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
# 
# y_pred = grid_search.predict(X_test)
# 
# y_test = y_test.to_numpy()
# # Debug: Check the shapes and types of y_pred
# print(f'y_pred: {y_pred}')
# print(f'y_test: {y_test}')
# 
# # Generate and display classification report
# print("Classification Report:")
# print(classification_report(y_test, y_pred))
# 
# # Generate and display confusion matrix
# cm = confusion_matrix(y_test, y_pred, labels=grid_search.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid_search.classes_)
# disp.plot(cmap=plt.cm.Blues)
# plt.show()
# 
